In [38]:
from py2neo import Graph, Node, Relationship

In [39]:
g = Graph('http://localhost:7474', auth=('neo4j', '123456'), name='test')

def create_graph():
    g.delete_all()

    g0 = Node('派别', name='明教')
    g.create(g0)
    g1 = Node('派别', name='武当')
    g.create(g1)
    g2 = Node('派别', name='峨眉')
    g.create(g2)
    g3 = Node('派别', name='天鹰教')
    g.create(g3)
    p0 = Node('男性人物', name='张三丰', 战斗力='999')
    g.create(p0)
    p1 = Node('男性人物', name='张无忌', 战斗力='998')
    g.create(p1)
    p2 = Node('男性人物', name='杨逍', 战斗力='500')
    g.create(p2)
    p3 = Node('男性人物', name='张翠山', 战斗力='300')
    g.create(p3)
    p4 = Node('女性人物', name='殷素素', 战斗力='200')
    g.create(p4)
    p5 = Node('男性人物', name='金毛狮王', 战斗力='600')
    g.create(p5)
    p6 = Node('女性人物', name='周芷若', 战斗力='900')
    g.create(p6)
    p7 = Node('女性人物', name='小昭', 战斗力='100')
    g.create(p7)
    p8 = Node('女性人物', name='蛛儿', 战斗力='60')
    g.create(p8)

    g.create(Relationship(p0, '属于', g1))
    g.create(Relationship(p3, '属于', g1))
    g.create(Relationship(p1, '属于', g0))
    g.create(Relationship(p2, '属于', g0))
    g.create(Relationship(p4, '属于', g3))
    g.create(Relationship(p5, '属于', g0))
    g.create(Relationship(p6, '属于', g2))
    g.create(Relationship(p8, '属于', g3))

    g.create(Relationship(p1, '父亲', p3))
    g.create(Relationship(p1, '母亲', p4))
    g.create(Relationship(p1, '义父', p5))
    g.create(Relationship(p1, '师公', p0))
    g.create(Relationship(p1, '情人', p7))
    g.create(Relationship(p1, '情人', p8))
    g.create(Relationship(p1, '手下', p2))

    g.create(Relationship(p6, '情敌', p7, level='非常敌视', Hatred=7000))
    g.create(Relationship(p7, '情敌', p6, level='非常敌视', Hatred=8000))
    g.create(Relationship(p6, '情敌', p8, level='敌视', Hatred=4000))
    g.create(Relationship(p8, '情敌', p6, Hatred=100))
    g.create(Relationship(p8, '情敌', p7, level='轻微敌视', Hatred=2000))
    g.create(Relationship(p7, '情敌', p8, level='非常敌视', Hatred=9000))

    g.create(Relationship(p3, '妻子', p4))


create_graph()

In [40]:
cql_nw = """MATCH (p:男性人物)
RETURN p"""
g.run(cql_nw).to_data_frame()

,p
0,"{'战斗力': '999', 'name': '张三丰'}"
1,"{'战斗力': '998', 'name': '张无忌'}"
2,"{'战斗力': '500', 'name': '杨逍'}"
3,"{'战斗力': '300', 'name': '张翠山'}"
4,"{'战斗力': '600', 'name': '金毛狮王'}"


In [41]:
# where子句
cql_w = """MATCH (p:男性人物)
WHERE p.name='张翠山'
RETURN p"""
g.run(cql_w).to_data_frame()

,p
0,"{'战斗力': '300', 'name': '张翠山'}"


In [42]:
# 布尔运算符:or  and   not
# 比较运算符:>    <    =    >=    <=    <>(不等于)
cql_w = """MATCH (p:男性人物)
WHERE p.name='张翠山' or p.name='张无忌'
RETURN p"""
g.run(cql_w).to_data_frame()

,p
0,"{'战斗力': '998', 'name': '张无忌'}"
1,"{'战斗力': '300', 'name': '张翠山'}"


In [43]:
# 创建两个现有节点之间的关系
# 节点名称(如:p1)字符长度若>=2,创建关系时使用括号包括(如:(p1))
cql_rw = """MATCH (p:男性人物), (p1:男性人物)
WHERE p.name='张翠山' and p1.name='张三丰'
create (p)-[r:师傅]->(p1)
RETURN p, p1"""
g.run(cql_rw).to_data_frame()

,p,p1
0,"{'战斗力': '300', 'name': '张翠山'}","{'战斗力': '999', 'name': '张三丰'}"


In [44]:
# 字符串正则表达式匹配(=~)
g.run("""MATCH (p:男性人物)
WHERE p.name=~'张.*'
RETURN p""").to_data_frame()

,p
0,"{'战斗力': '999', 'name': '张三丰'}"
1,"{'战斗力': '998', 'name': '张无忌'}"
2,"{'战斗力': '300', 'name': '张翠山'}"
